In [1]:
import os
import pandas as pd
import glob
from PIL import Image

In [2]:
# Chemin du répertoire contenant les images
repertoire = r'D:\aprojet\fev24_cds_plants\data\color'

# Liste pour stocker les informations sur les images
donnees = []

# Parcours des sous-répertoires et des fichiers
for dossier, sous_dossiers, fichiers in os.walk(repertoire):
    print("Parcours du répertoire :", dossier)
    for nom_fichier in glob.glob(os.path.join(dossier, '*.jpg')):
        chemin_image = nom_fichier
        classe = os.path.basename(dossier)
        nom_image = os.path.basename(nom_fichier)

        try:
            # Lecture de l'image avec Pillow
            image = Image.open(chemin_image)

            # Récupération de la largeur et de la hauteur
            largeur, hauteur = image.size

        except Exception as e:
            # Gestion des erreurs de lecture d'image
            print(f"Erreur lors de la lecture de l'image {chemin_image}: {e}")
            largeur = None
            hauteur = None

        donnees.append((chemin_image, classe, nom_image, largeur, hauteur))

# Création du dataframe
df = pd.DataFrame(donnees, columns=['Chemin', 'Classe', 'Nom_image', 'Largeur', 'Hauteur'])



Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Apple___Apple_scab
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Apple___Black_rot
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Apple___Cedar_apple_rust
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Apple___healthy
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Blueberry___healthy
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Cherry_(including_sour)___healthy
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Cherry_(including_sour)___Powdery_mildew
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Corn_(maize)___Common_rust_
Parcours du répertoire : D:\aprojet\fev24_cds_plants\data\color\Corn_(maize)___healthy
Parcours du répertoir

In [3]:
df.to_csv('dataset_plantdisease.csv', index=False)

In [5]:
df.head(10)

,Chemin,Classe,Nom_image,Largeur,Hauteur
0,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,00075aa8-d81a-4184-8541-b692b78d398a___FREC_Sc...,256,256
1,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Sc...,256,256
2,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,01f3deaa-6143-4b6c-9c22-620a46d8be04___FREC_Sc...,256,256
3,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,0208f4eb-45a4-4399-904e-989ac2c6257c___FREC_Sc...,256,256
4,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,023123cb-7b69-4c9f-a521-766d7c8543bb___FREC_Sc...,256,256
5,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,0261a6e4-21f8-481a-8827-b674e6955644___FREC_Sc...,256,256
6,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,029424b0-0ef5-491b-9ef5-069190d24d8f___FREC_Sc...,256,256
7,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,03354abb-aa1c-4f9d-a1ef-9f40505cd539___FREC_Sc...,256,256
8,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,0340dc35-5215-48ab-8db7-06af99fcb358___FREC_Sc...,256,256
9,D:\aprojet\fev24_cds_plants\data\color\Apple__...,Apple___Apple_scab,0395b847-2c73-4674-826f-33a6afb5b4fe___FREC_Sc...,256,256
